# **Memory & Chain**

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key[:5])

sk-pr


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [ ]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키'
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Memory**

### (1) 대화 기록 저장하기

In [3]:
# 메모리 선언하기(초기화)
memory = ConversationBufferMemory(return_messages=True)

# 저장
memory.save_context({"input": "안녕하세요!"},
                    {"output": "안녕하세요! 어떻게 도와드릴까요?"})

memory.save_context({"input": "메일을 써야하는데 도와줘"},
                    {"output": "누구에게 보내는 어떤 메일인가요?"})

# 현재 담겨 있는 메모리 내용 전체 확인
memory.load_memory_variables({})

{'history': [HumanMessage(content='안녕하세요!'),
  AIMessage(content='안녕하세요! 어떻게 도와드릴까요?'),
  HumanMessage(content='메일을 써야하는데 도와줘'),
  AIMessage(content='누구에게 보내는 어떤 메일인가요?')]}

### (2) GPT와 대화 기록 저장하기

#### 1) 대화 준비

In [4]:
k = 3

# Chroma 데이터베이스 인스턴스
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = "./data3", embedding_function = embeddings)

# retriever 선언
retriever = database.as_retriever(search_kwargs={"k": k})

# ChatOpenAI 선언
chat = ChatOpenAI(model="gpt-3.5-turbo")

# RetrievalQA 선언
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True)

#### 2) 대화 시도 및 기록

In [5]:
# 질문 답변1
query = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"
result = qa(query)
result['result']

'생성형 AI 도입시 예상되는 보안 위협은 다양합니다. 몇 가지 예시로는 악의적인 공격자가 프로젝트 팀에 침투하여 소프트웨어에 악성 코드를 추가하거나, 훈련 데이터나 모델을 오염시키는 것이 있습니다. 또한 해커가 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스와 잘못된 정보로 모델을 훈련시키는 등의 위협도 있을 수 있습니다. 생성형 AI 프로젝트의 복잡성과 다양성으로 인해 보안 위협이 다른 소프트웨어보다 더 많이 발생할 수 있습니다.'

In [6]:
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 도입시 예상되는 보안 위협은 다양합니다. 몇 가지 예시로는 악의적인 공격자가 프로젝트 팀에 침투하여 소프트웨어에 악성 코드를 추가하거나, 훈련 데이터나 모델을 오염시키는 것이 있습니다. 또한 해커가 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스와 잘못된 정보로 모델을 훈련시키는 등의 위협도 있을 수 있습니다. 생성형 AI 프로젝트의 복잡성과 다양성으로 인해 보안 위협이 다른 소프트웨어보다 더 많이 발생할 수 있습니다.')]}

#### 3) 이어지는 질문과 답변

In [7]:
# 질문 답변2
query = "훈련 데이터나 가중치를 오염시키는게 무슨 의미야?"
result = qa(query)
result['result']

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 도입시 예상되는 보안 위협은 다양합니다. 몇 가지 예시로는 악의적인 공격자가 프로젝트 팀에 침투하여 소프트웨어에 악성 코드를 추가하거나, 훈련 데이터나 모델을 오염시키는 것이 있습니다. 또한 해커가 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스와 잘못된 정보로 모델을 훈련시키는 등의 위협도 있을 수 있습니다. 생성형 AI 프로젝트의 복잡성과 다양성으로 인해 보안 위협이 다른 소프트웨어보다 더 많이 발생할 수 있습니다.'),
  HumanMessage(content='훈련 데이터나 가중치를 오염시키는게 무슨 의미야?'),
  AIMessage(content='훈련 데이터나 가중치를 오염시킨다는 것은 악의적인 공격자가 AI 모델을 속이기 위해 훈련 데이터나 모델 내부의 가중치를 조작하여 모델의 성능을 악화시키거나 의도치 않은 행동을 유발하도록 하는 것을 의미합니다. 이는 모델의 신뢰성을 훼손하고 사용자의 데이터 또는 시스템을 위협할 수 있는 보안 위협으로 작용할 수 있습니다.')]}

In [8]:
# 질문 답변3
query = "이를 방지하기 위해 어떻게 해야 해?"
result = qa(query)
result['result']

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 도입시 예상되는 보안 위협은 다양합니다. 몇 가지 예시로는 악의적인 공격자가 프로젝트 팀에 침투하여 소프트웨어에 악성 코드를 추가하거나, 훈련 데이터나 모델을 오염시키는 것이 있습니다. 또한 해커가 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스와 잘못된 정보로 모델을 훈련시키는 등의 위협도 있을 수 있습니다. 생성형 AI 프로젝트의 복잡성과 다양성으로 인해 보안 위협이 다른 소프트웨어보다 더 많이 발생할 수 있습니다.'),
  HumanMessage(content='훈련 데이터나 가중치를 오염시키는게 무슨 의미야?'),
  AIMessage(content='훈련 데이터나 가중치를 오염시킨다는 것은 악의적인 공격자가 AI 모델을 속이기 위해 훈련 데이터나 모델 내부의 가중치를 조작하여 모델의 성능을 악화시키거나 의도치 않은 행동을 유발하도록 하는 것을 의미합니다. 이는 모델의 신뢰성을 훼손하고 사용자의 데이터 또는 시스템을 위협할 수 있는 보안 위협으로 작용할 수 있습니다.'),
  HumanMessage(content='이를 방지하기 위해 어떻게 해야 해?'),
  AIMessage(content='악의적인 공격을 방지하기 위해서는 다음과 같은 조치를 취할 수 있습니다:\n1. 오픈소스 모델의 사용시 주의: 오픈소스 모델을 사용할 때는 신중하게 선택하고, 신뢰할 수 있는 소스에서 다운로드 받아야 합니다.\n2. 보안 전문가의 참여: 기업에서는 오픈소스 AI 모델 관련 보안 전문가를 고용하여 보안 취약점을 식별하고 예방할 수 있습니다.\n3. 모델 테스트 및 검증: 모델을 정기적으로 테스트하고 검증하여 취약점을 식별하고 보완해야 합니다.\n4. 보안 강화: 데이터의 보안을 강화하고, 접근 권한을 제한하여 악의적인 공격을 방지할 수 있습니다.\n5. 고위험 오픈소스 모델 회피: 

* 훈련 데이터 오염을 방지하기 위한 대책을 물었으나, 일반적인 보안 위협 방지 대책을 이야기 함.
* 맥락을 유지하기 위해서 메시지의 내용을 프롬프트에 포함시켜야 함.
    * 이를 손쉽게 엮어주는 방법 **Chain**

## **3.Chain**

* 절차 다시 정리
    * 질문을 받아
    * 유사도 높은 문서를 DB에서 검색(RAG)
    * 이전 대화 내용을 메모리에서 읽어오기
    * [질문 + 유사도높은 문서 + 이전 대화내용]으로 프롬프트 구성
    * GPT에 질문하고 답변 받기
* Chain
    * 이러한 절차를 코드로 하나하나 엮고, 프롬프트를 구성하는 코드는 상당히 복잡합니다.
    * 랭체인에서 제공하는 Chain 함수를 이용하면 쉽게 구현 가능!  

### (1) Chain 함수로 연결하기
* **ConversationalRetrievalChain**

In [9]:
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = "./data3", embedding_function = embeddings)
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=3
retriever = database.as_retriever(search_kwargs={"k": k})

# 대화 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", 
                                  output_key="answer", return_messages=True)

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory, # memory 지정
                                           return_source_documents=True,  output_key="answer") # output_key, memory의 값과 맞춰준다

### (2) 사용하기

#### 1) 첫번째 질문

In [10]:
# 첫번째 질문
query1 = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"
result = qa(query1)
result['answer']

'생성형 AI 프로젝트는 악의적인 공격을 받을 수 있는 여러 위험 요소가 있습니다. 악의적인 개발팀이 프로젝트에 침투하여 악성 코드를 추가하거나 훈련 데이터, 미세 조정, 가중치를 오염시키는 경우가 있을 수 있습니다. 또한 해커가 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스 및 잘못된 정보로 모델을 훈련시킬 수도 있습니다. 이러한 위협을 경감하기 위해 안전성 평가 및 보호 장치 마련이 필요합니다.'

In [11]:
# 메모리 확인
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 프로젝트는 악의적인 공격을 받을 수 있는 여러 위험 요소가 있습니다. 악의적인 개발팀이 프로젝트에 침투하여 악성 코드를 추가하거나 훈련 데이터, 미세 조정, 가중치를 오염시키는 경우가 있을 수 있습니다. 또한 해커가 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스 및 잘못된 정보로 모델을 훈련시킬 수도 있습니다. 이러한 위협을 경감하기 위해 안전성 평가 및 보호 장치 마련이 필요합니다.')]}

#### 2) 두번째 질문

In [12]:
# 두번째 질문
query2 = "모델을 재학습시키면 어떤 문제가 발생되는거야?"
result = qa(query2)
result['answer']

'모델을 재학습시키면 해커가 악의적인 목적을 달성하기 위해 모델을 오염시킬 수 있습니다. 이를 통해 해커는 사용자 인프라에 침입하거나 가짜 뉴스 및 잘못된 정보로 모델을 훈련시킬 수 있습니다. 이러한 공격은 모델의 신뢰성을 저하시키고 전체 시스템에 심각한 피해를 줄 수 있습니다.'

In [13]:
# 메모리 확인
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI 프로젝트는 악의적인 공격을 받을 수 있는 여러 위험 요소가 있습니다. 악의적인 개발팀이 프로젝트에 침투하여 악성 코드를 추가하거나 훈련 데이터, 미세 조정, 가중치를 오염시키는 경우가 있을 수 있습니다. 또한 해커가 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스 및 잘못된 정보로 모델을 훈련시킬 수도 있습니다. 이러한 위협을 경감하기 위해 안전성 평가 및 보호 장치 마련이 필요합니다.'),
  HumanMessage(content='모델을 재학습시면 어떤 문제가 발생되는거야?'),
  AIMessage(content='모델을 재학습시키면 해커가 악의적인 목적을 달성하기 위해 모델을 오염시킬 수 있습니다. 이를 통해 해커는 사용자 인프라에 침입하거나 가짜 뉴스 및 잘못된 정보로 모델을 훈련시킬 수 있습니다. 이러한 공격은 모델의 신뢰성을 저하시키고 전체 시스템에 심각한 피해를 줄 수 있습니다.')]}

### (3) 반복문 안에서 질문답변 이어가기

In [14]:
while True:
    query = input('질문 > ')
    query = query.strip()
    print(f'질문 : {query}')
    print('-' * 20)
    if len(query) == 0:
        break
    result = qa({"question": query})
    print(f'답변 : {result["answer"]}')
    print('=' * 50)

질문 : 어떤 보안 위협이 있어?
--------------------
답변 : 생성형 AI 프로젝트에서 발생할 수 있는 보안 위협에는 여러 가지가 있습니다. 일부 위협에는 해커가 소프트웨어에 악성 코드를 추가하는 것, 훈련 데이터나 모델을 오염시키는 것, 모델을 재훈련시켜 인프라에 침입하는 것, 가짜 뉴스나 잘못된 정보로 모델을 훈련시키는 것, 오픈소스 모델을 사용하여 해킹을 시도하는 것, 모델의 시스템 프롬프트를 분석하여 취약점을 찾는 것 등이 있습니다. 또한, 모델 출력물에 워터마크를 추가해도 공격을 막지 못할 수 있고, 특정 요청을 계속 보내 인프라를 과부하시켜 모델이 작동하지 않도록 만드는 것 또한 위협이 될 수 있습니다.
질문 : 악의적인 공격이라는게 무슨 의미야?
--------------------
답변 : 악의적인 공격이란 사이버 보안 관련 용어로, 해커 또는 사이버 공격자가 악의적인 목적을 가지고 컴퓨터 시스템, 네트워크, 소프트웨어 등에 침입하여 정보를 도용하거나 파괴하거나 다른 해로운 행위를 하는 것을 말합니다. 이는 주로 개인 정보 도용, 시스템 마비, 악성 코드 삽입, 데이터 변조 등을 포함할 수 있습니다.
질문 : 선의의 공격도 있어?
--------------------
답변 : 선의의 공격은 주로 시스템의 취약점을 발견하고 보고하는 행위를 말합니다. 이는 보안 취약점을 개선하여 시스템을 더 안전하게 만드는 데 도움이 됩니다. Seungkyu라고 언급한 대로 선의의 공격은 악의가 아니라 시스템의 보안을 강화하기 위한 목적으로 이루어집니다.
질문 : 
--------------------
